In [3]:
import requests
import pandas as pd

# Example: Get current F1 race data
url = "http://ergast.com/api/f1/drivers/lewis.json"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    new_data = pd.DataFrame(data)
    print(new_data)
else:
    print("Error fetching data from Ergast API")
    print("")


                                                        MRData
xmlns                                http://ergast.com/mrd/1.5
series                                                      f1
url                http://ergast.com/api/f1/drivers/lewis.json
limit                                                       30
offset                                                       0
total                                                        1
DriverTable  {'driverId': 'lewis', 'Drivers': [{'driverId':...
